# GNN solver for prediction of power flow
This notebooks allows to train a Physics Informed GNN as shown in figures below. A MLP neural network is used to initiate the voltage angles, thereafter, a sucession of GNN Message Passing mechanism and Local Consevation (LC) layers allows the computation of power grid equations and errors.

<div style="text-align: center;">
<img src="../imgs/gnn_eq_powergrid.png" width=400 height=70 />
</div>

<div style="text-align: center;">
<img src="../imgs/gnn_scheme_powergrid.png" width=600 height=250 />
</div>

In [1]:
import sys
sys.path.append("../.")

In [2]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import torch
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from gnn_powergrid.dataset import prepare_dataset

## Prepare the dataset

In [3]:
env_name = "l2rpn_case14_sandbox"

path = pathlib.Path().resolve().parent
BENCH_CONFIG_PATH = path / "configs" / (env_name + ".ini")
DATA_PATH = path / "Datasets" / env_name / "DC"
LOG_PATH = path / "lips_logs.log"

benchmark = PowerGridBenchmark(benchmark_path=DATA_PATH,
                               benchmark_name="Benchmark4",#"DoNothing",
                               load_data_set=True,
                               config_path=BENCH_CONFIG_PATH,
                               log_path=LOG_PATH)

In [ ]:
# Creating data loaders for GNN. This step is integrated as data processing in GNN
# device = torch.device("cpu") # or "cuda:0" if you have any GPU
# train_loader, val_loader, test_loader, test_ood_loader = prepare_dataset(benchmark=benchmark, 
#                                                                          batch_size=128, 
#                                                                          device=device)
# batch = next(iter(test_loader))
# print(batch)

## Initialize the GNN solver

In [47]:
from gnn_powergrid.gnn.gnn_simulator import GnnSimulator
from gnn_powergrid.gnn.models.gnn import GPGmodel

In [48]:
SIM_CONFIG_PATH = path / "configs" / "gnn.ini"
gnn_simulator = GnnSimulator(model=GPGmodel,
                             name="gnn_torch",
                             sim_config_path=SIM_CONFIG_PATH,
                             input_size=2,
                             output_size=1,
                             epochs=2)

In [49]:
gnn_simulator._model.params

{'env_name': 'l2rpn_case14_sandbox',
 'name': 'torch_gnn',
 'ref_node': 0,
 'num_gnn_layers': 10,
 'latent_dimension': 20,
 'hidden_layers': 3,
 'input_dim': 2,
 'output_dim': 1,
 'train_batch_size': 128,
 'eval_batch_size': 128,
 'device': 'cpu',
 'optimizer': {'name': 'adam', 'params': {'lr': 0.0003}},
 'epochs': 2,
 'train_with_discount': False,
 'save_freq': False,
 'ckpt_freq': 50,
 'scaler': None,
 'input_size': 2,
 'output_size': 1}

### Train the GNN solver

In [ ]:
gnn_simulator.train(benchmark.train_dataset, benchmark.val_dataset)

### Get the predictions on a specific dataset

In [ ]:
predictions = gnn_simulator.predict(dataset=benchmark._test_dataset)

### Evaluate the GNN solver using LIPS evaluation

In [ ]:
metrics_test = benchmark.evaluate_simulator(dataset="all", augmented_simulator=gnn_simulator)

In [55]:
from pprint import pprint
pprint(metrics_test)

{'test': {'ML': {'MAE_avg': {'p_ex': 1.9286174774169922,
                             'p_or': 1.9286174774169922},
                 'MAPE_10_avg': {'p_ex': 0.14798755201207694,
                                 'p_or': 0.14798755201207694},
                 'MAPE_90_avg': {'p_ex': 0.1526969598806569,
                                 'p_or': 0.1526969598806569},
                 'MAPE_avg': {'p_ex': 79694225473536.0,
                              'p_or': 79694225473536.0},
                 'MSE_avg': {'p_ex': 9.320919036865234,
                             'p_or': 9.320919036865234}},
          'Physics': {'CHECK_GC': {'mae': 1.0942078e-05,
                                   'violation_percentage': 0.0,
                                   'wmape': 1.0},
                      'CHECK_LC': {'mae': 1.9472226934313903,
                                   'mape': 0.0932005681107784,
                                   'violation_percentage': 99.67571428571429},
                      'CHECK_LOSS':